In [36]:
# 144 tiles (12x12)
# sides are length 10, so there are 1024 possibilities
# In reality, there are ~144*4/2 = 288 different sides (slightly more because of edges don't have to match)
# Also, it's multiplied by 2 because they can be flipped
# So it is worth checking if the edges are indeed unique.

# If so that would make it doable without any brute force, since we could essentially start with an arbitrary piece,
# and the entire puzzle would be 'forced' into place
import numpy as np
import math

In [37]:
num_tiles = 144
tile_size = 10

with open("data/advent20.txt", "r") as f:
    data = f.read().splitlines()

tiles = [data[(i+1):(i+11)] for i in list(range(0, num_tiles*(tile_size+2), tile_size+2))]
names = [int(data[i][5:9]) for i in list(range(0, num_tiles*(tile_size+2), tile_size+2))]  # assume names are always 4 digits

In [44]:
def get_edges(tile):
    edges = [tile[0], tile[-1], ''.join(t[0] for t in tile), ''.join(t[-1] for t in tile)]
    flipped = [edge[::-1] for edge in edges]
    return edges + flipped

all_edges = []
for tile in tiles:
    all_edges += get_edges(tile)

# Check if any edges appear more than twice:
real_edges = []
for e in all_edges:
    if all_edges.count(e) > 2:
        print(e)
    elif all_edges.count(e) == 1:
        real_edges.append(e)

# No output is a good sign!
print(len(real_edges))  # This should be 96, for the 12*4 edges that don't have a match, *2 for flip

96


In [45]:
# Find corners: puzzles with two pieces in 'real_edges'
result = 1
corner_ix = None  # A corner. Used for part 2 as top left corner
for ix, tile in enumerate(tiles):
    edges = get_edges(tile)
    real = sum([e in real_edges for e in edges])
    if real == 4:
        print(ix, names[ix])
        corner_ix = ix
        result *= names[ix]
print(result)

10 3833
46 2593
71 2999
76 3517
104831106565027


In [116]:
def to_numpy(tile):
    # Is idempotent
    tile = [[c for c in row] for row in tile]
    return np.array(tile)

def to_python(tile):
    return ''.join(tile)

def remove_edge(tile, make_numpy=True):
    result = [r[1:-1] for r in tile[1:-1]]
    return to_numpy(result) if make_numpy else result

def rotate_py(tile):
    # Given a python tile (list of strings), rotate it clockwise 90 degrees
    return [''.join(t[i] for t in tile[::-1]) for i in range(len(tile))]

def flip_py(tile):
    # Given a python tile, flip it horizontally
    return [row[::-1] for row in tile]

def rotate_topleft_corner(tile, edges):
    # Given a corner piece, rotate it so that it's oriented correctly
    # Note that the final puzzle may also need to be rotated/flipped still
    es = get_edges(tile)  # Top, bottom, left, right
    if edges.count(es[0]) == 1 and edges.count(es[2]) == 1:
        return tile  # It's already correct
    else:
        # Rotate and try again
        return rotate_topleft_corner(rotate_py(tile), edges)

def find_tile_with_edge(edge, tiles):
    # Possible optimization: use a dict for the edges instead of a function
    # Since we recalculate get_edges a lot... But it's not that bad
    # Also returns the orientation of that edge in the found tile:
    # 0 - top, 1 - bottom, 2 - left, 3 - right
    # 4 - topflipped, 5 - bottomflipped, 6 - leftflipped, 7 - rightflipped
    for ix, tile in enumerate(tiles):
        if edge in get_edges(tile):
            return ix, get_edges(tile).index(edge)
    raise ValueError("{} could not be found".format(edge))

def match_left(tile, match):
    # Given a python tile that matches the edge on the left, (from top to bottom)
    # and a match (from find_tile_with_edge)
    # rotate the tile so that it fits the edge on the left
    if match == 0:
        return rotate_py(rotate_py(rotate_py(flip_py(tile))))
    elif match == 1:
        return rotate_py(tile)
    elif match == 2:
        return tile
    elif match == 3:
        return flip_py(tile)
    elif match == 4:
        return rotate_py(rotate_py(rotate_py(tile)))
    elif match == 5:
        return rotate_py(flip_py(tile))
    elif match == 6:
        return rotate_py(rotate_py(flip_py(tile)))
    elif match == 7:
        return rotate_py(rotate_py(tile))

def match_top(tile, match):
    # Same as match_left, but rotate the tile such that it matches the edge on top
    # (from left to right)
    # flip(rotate(x)) is basically transpose(x)
    return flip_py(rotate_py(match_left(tile, match)))

def construct_puzzle(tiles, corner_tile):
    # Given a list of tiles, and the corner tile, constructs a square puzzle
    # Tiles must not contain the corner_tile
    # Corner tile must have been rotated with rotate_topleft_corner
    puzzle_size =  int(math.sqrt(len(tiles) + 1))
    piece_size = len(corner_tile)  # Must be square.

    puzzle = np.empty((puzzle_size*piece_size, puzzle_size*piece_size), dtype=str)
    puzzle[:piece_size, :piece_size] = to_numpy(corner_tile)

    # Now we have the first tile, we can start filling in the rest one by one
    # Remember: x = up/down, y = left/right
    for tile_y in range(puzzle_size):
        for tile_x in range(puzzle_size):
            if tile_x == 0 and tile_y == 0:
                continue  # We already did this one as initialization
            elif tile_x == 0:
                # The first in a row. We need to compare the top edge instead of the left edge
                # x coordinate: -1, 11, 23, etc = (tile_x * tile_size) - 1
                # y coordinate: 0 to 11
                # This is the tile from top to bottom
                edge = puzzle[(tile_y*tile_size)-1, :tile_size]
                match_fn = match_top
            else:
                # y coordinate: -1, 11, 23, etc = (tile_x * tile_size) - 1
                # x coordinate: 0 to 11, 12 to 23, 23 to 34, etc
                # This is the tile from top to bottom
                edge = puzzle[(tile_y*tile_size):((tile_y+1)*tile_size), (tile_x*tile_size)-1]
                match_fn = match_left

            #print(tile_x, tile_y, edge, to_python(edge), (tile_y*tile_size), ((tile_y+1)*tile_size), (tile_x*tile_size)-1)
            
            x = find_tile_with_edge(to_python(edge), tiles)
            tile_ix, match = x[0], x[1]
            tile = match_fn(tiles[tile_ix], match)
            tiles.pop(tile_ix)  # Remove it so it doesn't bother us again
            puzzle[(tile_y*tile_size):((tile_y+1)*tile_size), (tile_x*tile_size):((tile_x+1)*tile_size)] = to_numpy(tile)
    return puzzle

corner_tile = rotate_topleft_corner(tiles[corner_ix], all_edges)
tiles_without_corner = tiles.copy()
tiles_without_corner.pop(corner_ix)

puzzle = construct_puzzle(tiles_without_corner, corner_tile)
#puzzle


In [146]:
def remove_edges_from_solved(puzzle, piece_size):
    # Remove edges back to front. First axis=0, then axis=1
    puzzle_size = puzzle.shape[0]
    num_pieces = int(puzzle_size / piece_size)
    #edges = [120, 111, 110, ..., 11, 10, 1]
    edges = sorted(list(range(0, puzzle_size, piece_size)) + list(range(piece_size-1, puzzle_size, piece_size)), reverse=True)
    print(edges, len(edges))
    for e in edges:
        puzzle = np.delete(puzzle, e, 0)
        puzzle = np.delete(puzzle, e, 1)
    return puzzle

puzzle_final = remove_edges_from_solved(puzzle, 10)

[119, 110, 109, 100, 99, 90, 89, 80, 79, 70, 69, 60, 59, 50, 49, 40, 39, 30, 29, 20, 19, 10, 9, 0] 24


In [152]:
def is_monster(image):
    # Image must be a 3x20 array. Works both on numpy and list
    return (image[0][18] == '#') and \
           (image[1][0]  == '#') and \
           (image[1][5]  == '#') and \
           (image[1][6]  == '#') and \
           (image[1][11]  == '#') and \
           (image[1][12]  == '#') and \
           (image[1][17]  == '#') and \
           (image[1][18]  == '#') and \
           (image[1][19]  == '#') and \
           (image[2][1]  == '#') and \
           (image[2][4]  == '#') and \
           (image[2][7]  == '#') and \
           (image[2][10]  == '#') and \
           (image[2][13]  == '#') and \
           (image[2][16]  == '#')


def count_monsters(puzzle_final, monster_size = (3, 20)):
    monsters = []
    for y in range(puzzle_final.shape[0] - monster_size[1]):
        for x in range(puzzle_final.shape[1] - monster_size[0]):
            monsters.append(is_monster(puzzle_final[x:(x+monster_size[0]), y:(y+monster_size[1])]))
    return sum(monsters)

monsters_count = []
monsters_count.append(count_monsters(puzzle_final))
monsters_count.append(count_monsters(np.rot90(puzzle_final)))
monsters_count.append(count_monsters(np.rot90(np.rot90(puzzle_final))))
monsters_count.append(count_monsters(np.rot90(np.rot90(np.rot90(puzzle_final)))))
monsters_count.append(count_monsters(np.transpose(puzzle_final)))
monsters_count.append(count_monsters(np.rot90(np.transpose(puzzle_final))))
monsters_count.append(count_monsters(np.rot90(np.rot90(np.transpose(puzzle_final)))))
monsters_count.append(count_monsters(np.rot90(np.rot90(np.rot90(np.transpose(puzzle_final))))))
print(monsters_count)


[0, 0, 0, 37, 0, 0, 0, 0]


In [157]:
# At this point I should have used a dictionary to automatically get the correct rotation, but I didn't feel it was worth it
# And this has taken long enough. So I get it manually...
puzzle_final_final = np.rot90(np.rot90(np.rot90(puzzle_final)))
sum(puzzle_final_final.flatten() == '#') - (37*15)  # 37 sea monsters, 15 # each

2093